# CAST RECOGNITION

In [ ]:
!pip install face_recognition

In [ ]:
import face_recognition as fr
import matplotlib.pyplot as plt
import os
import cv2
import face_recognition
import numpy as np
from time import sleep
import PIL.Image
import PIL.ImageDraw

def get_encoded_faces():
    """
    :return: dict of (name, image encoded)
    """
    encoded = {}
    i = 1
    print("Detecting face...")
    for dirpath, dnames, fnames in os.walk("../input/my-train/"):
        for f in fnames:
            if f.endswith(".jpg") or f.endswith(".png"):
                face = fr.load_image_file("../input/my-train/" + f)
                landmarks = fr.face_landmarks(face)
                if(f == "Gwyneth_Paltrow.jpg"):
                    print("\nThe face landmarks for "+ '{}'.format(f)+' are:')
                    print(landmarks)
                    print("\n")
                #print(i,"==>",end ="")
                
                if not landmarks:
                    break
                else:
                    encoding = fr.face_encodings(face)[0]
                    encoded[f.split(".")[0]] = encoding
                    if(f == "Gwyneth_Paltrow.jpg"):
                        print("Face-encoded values for "+'{}'.format(f)+" are: ")
                        print(encoded[f.split(".")[0]])
                i = i+1
                   
    return encoded

In [ ]:
def classify_face(im):
    """
    will find all of the faces in a given image and label
    them if it knows what they are

    :param im: str of file path
    :return: list of face names
    """
    faces = get_encoded_faces()
    faces_encoded = list(faces.values())
    known_face_names = list(faces.keys())
    
    img = cv2.imread(im, 1)
    
    face_locations = face_recognition.face_locations(img)
    unknown_face_encodings = face_recognition.face_encodings(img, face_locations)

    face_names = []
    for face_encoding in unknown_face_encodings:
        matches = face_recognition.compare_faces(faces_encoded, face_encoding)
        name = "Cast not found"
        face_distances = face_recognition.face_distance(faces_encoded, face_encoding)
        print()
        #for i, face_distance in enumerate(face_distances):
            #print("The test image has a distance of {:.2} from known image #{}".format(face_distance, i))
            #print("- With a very strict cutoff would the test image match the known image? {}".format(face_distance < 0.5))
            #print()
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        face_names.append(name)
        pil_image = PIL.Image.fromarray(img)

        for (top, right, bottom, left), name in zip(face_locations, face_names):
            cv2.rectangle(img, (left-20, top-20), (right+20, bottom+20), (255, 0, 0), 2)
            cv2.rectangle(img, (left-20, bottom -15), (right+20, bottom+20), (255, 0, 0), cv2.FILLED)
    plt.figure(figsize = (5,5))
    plt.rcParams["axes.grid"] = False
    plt.imshow(img) 
    plt.show()
    return face_names 

In [ ]:

def final(path):
    print("The cast in the movie poster are:")
    print(cast)
    print()
    prediction(path)

In [ ]:
test_path = '../input/test-data/img_15.jpg'
cast = classify_face(test_path)

### PERFORMANCE METRICS

In [ ]:
from PIL import Image
labels_face = []
y_test_face = []
ypred_face = []
i = 0
for files in os.listdir('../input/acc-test'):
    labels_face.append(files)
    img = Image.open('../input/acc-test/' + files)
    y_test_face.append(i)
    i = i+1
    ypred_face.append(classify_face('../input/acc-test/' + files))
print(y_test_face)
print(labels_face)
print(ypred_face)

In [ ]:
print(ypred_face)
print(labels_face)

In [ ]:
ypreds = np.array(ypred_face).reshape(1,-1)[0]

In [ ]:
import pandas as pd
y_pred_mapped = pd.Series(ypreds).map({'Ajay_Devgan':0,'hansika_motwani':1,'Jayam_Ravi':2,'Aswin':3,'Brad_Pitt':4,'Abishek_Bacchan':5,'Aishwarya_rai':6}).astype(int)
print(y_pred_mapped)

In [ ]:
y_pred_print = pd.Series(ypreds)
y_pred_print

In [ ]:
from sklearn.metrics import accuracy_score
print("Acuuracy of the model:")
print(accuracy_score(y_pred_mapped,y_test_face)*100,"%")

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, classification_report
print(classification_report(y_pred_mapped, y_test_face))

In [ ]:
face_matrix = multilabel_confusion_matrix(y_test_face,y_pred_mapped)
display(face_matrix)
face_matrix[0]

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt

test_val = y_test_face
pred_val = y_pred_mapped
pred_val = np.round(pred_val)
c_matrix = confusion_matrix(test_val, pred_val)
df_cm = pd.DataFrame(c_matrix, columns=['Abishek_Bacchan','Jayam_Ravi','Aishwarya_rai','Hansika','Ajay_Devgan','Aswin','Brad'], index = ['Abishek_Bacchan','Jayam_Ravi','Aishwarya_rai','Hansika','Ajay_Devgan','Aswin','Brad'])
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
plt.figure(figsize = (5,5))
sn.set(font_scale=1.4) 
sn.heatmap(df_cm, cmap="PuRd", annot=True,fmt='',annot_kws={"size": 16})
plt.show()